In [10]:
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torch.utils import data
from torchvision import datasets
from torchvision.transforms import v2

In [11]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [12]:
class PUNKClassifier(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, (5,5), stride=1, padding=4),
            nn.Conv2d(128, 128, (3,3), stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, (5,5), stride=1, padding=4),
            nn.Conv2d(64, 64, (3,3), stride=2, padding=2),
            nn.ReLU(inplace=True)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(129600, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, output_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x):

        x = self.features(x)
        x = self.classifier(x)
        return x


In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
IMAGE_SIZE = 160

transformer = v2.Compose([
    v2.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float, scale=True),
    v2.ToTensor()
])

dataset = datasets.ImageFolder("../data", transform=transformer)

classes = dataset.classes

train_set, val_set, test_set = data.random_split(dataset, [0.7, 0.15, 0.15])

In [16]:
train_set.dataset.transform = train_transforms = v2.Compose([
    v2.RandomRotation(30),
    v2.RandomResizedCrop(IMAGE_SIZE),
    v2.RandomHorizontalFlip(),
    v2.ToTensor()
])

In [17]:
batch_size = 32

# Create data loaders.
train_dataloader = data.DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_dataloader = data.DataLoader(test_set, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 3, 160, 160])
Shape of y: torch.Size([32]) torch.int64


In [18]:
epochs = 50

model = PUNKClassifier(len(classes)).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------


: 

In [ ]:
torch.save(model.state_dict(), "../trained_models/ckpt")